In [39]:
import pandas as pd

In [40]:
df = pd.read_csv('/content/drive/MyDrive/datasets/twitter_training.csv')
df.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [41]:
df.isnull().sum()
df.dropna(axis=0)
df.rename(columns={"2401": "id"}, inplace=True)
df.rename(columns={"Positive": "sentiment"}, inplace=True)
df.rename(columns={"Borderlands": "user"}, inplace=True)
df.rename(columns={"im getting on borderlands and i will murder you all ,": "tweet"}, inplace=True)
df['tweet'] = df['tweet'].astype(str)

In [42]:
from sklearn.model_selection import train_test_split

X = df['tweet']
Y = pd.get_dummies(df['sentiment'])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [43]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

max_tokens = 64

tokenizer = Tokenizer(num_words=max_tokens)
tokenizer.fit_on_texts(X_train.values)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [45]:
maxlen = 16

X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [49]:
from keras.models import Sequential
import keras
from keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, LSTM, Flatten
from sklearn.metrics import confusion_matrix, classification_report

num_classes = len(Y.columns)

model = Sequential()
model.add(Embedding(max_tokens, 32, input_length=X_train.shape[1]))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax'))

In [50]:
model.compile(loss=keras.losses.CategoricalCrossentropy(), optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=[keras.metrics.Precision(), keras.metrics.Recall()])

In [51]:
history = model.fit(X_train, Y_train, validation_split=0.2, batch_size=64, epochs=10)

Epoch 1/10
747/747 [==============================] - 20s 20ms/step - loss: 1.3211 - precision: 0.5720 - recall: 0.0546 - val_loss: 1.2914 - val_precision: 0.5602 - val_recall: 0.1172
Epoch 2/10
747/747 [==============================] - 14s 18ms/step - loss: 1.2721 - precision: 0.5955 - recall: 0.1181 - val_loss: 1.2719 - val_precision: 0.6163 - val_recall: 0.1175
Epoch 3/10
747/747 [==============================] - 15s 20ms/step - loss: 1.2505 - precision: 0.6248 - recall: 0.1468 - val_loss: 1.2585 - val_precision: 0.6085 - val_recall: 0.1483
Epoch 4/10
747/747 [==============================] - 15s 20ms/step - loss: 1.2346 - precision: 0.6349 - recall: 0.1691 - val_loss: 1.2517 - val_precision: 0.6134 - val_recall: 0.1483
Epoch 5/10
747/747 [==============================] - 14s 19ms/step - loss: 1.2208 - precision: 0.6410 - recall: 0.1823 - val_loss: 1.2399 - val_precision: 0.6253 - val_recall: 0.1681
Epoch 6/10
747/747 [==============================] - 15s 20ms/step - loss: 1.20

In [52]:
preds = model.predict(X_test)

467/467 [==============================] - 3s 6ms/step


In [54]:
import numpy as np

print(classification_report(np.argmax(Y_test.values, axis=1), np.argmax(preds, axis=1)))

              precision    recall  f1-score   support

           0       0.42      0.09      0.15      2604
           1       0.49      0.50      0.50      4481
           2       0.53      0.41      0.46      3684
           3       0.40      0.67      0.50      4168

    accuracy                           0.45     14937
   macro avg       0.46      0.42      0.40     14937
weighted avg       0.46      0.45      0.43     14937

